[link code gốc](**https**://github.com/abdelghanibelgaid/nnvpp/blob/main/notebook.ipynb)

In [ ]:
import os, io, gc
import numpy as np
import pandas as pd
import random

from scipy.fft import fft
from scipy.signal import hilbert, blackman
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Dropout
from tensorflow.keras.layers import Add, concatenate, Dense, LSTM, GRU, SimpleRNN, RNN, Input, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential, load_model

import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)
plt.style.use('fivethirtyeight')
import itertools
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.model_selection import GroupKFold, train_test_split
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error , r2_score
import warnings
warnings.filterwarnings('ignore')
gc.enable()
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
SEED = 42
def seed_everything(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
seed_everything()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning đồ án/Ventilator-pressure dataset/data.csv')

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=7) 
test, val = train_test_split(test, test_size=0.5, random_state=7)
train=train.iloc[:train.shape[0]//80*80,:]
test=test.iloc[:test.shape[0]//80*80,:]
val = val.iloc[:val.shape[0]//80*80,:]
print(train.shape, test.shape, val.shape)

(362480, 92) (20080, 92) (20080, 92)


In [ ]:
y_train = train[['pressure']].to_numpy().reshape(-1, 80)
x_train = train.drop(['pressure','id', 'breath_id'], axis=1)
y_test = test[['pressure']].to_numpy().reshape(-1, 80)
x_test = test.drop(['id', 'breath_id', 'pressure'], axis=1)
y_val = val[['pressure']].to_numpy().reshape(-1, 80)
x_val = val.drop(['pressure','id', 'breath_id'], axis=1)

# scale
RS = RobustScaler(quantile_range=(20.0, 80.0))
x_train = RS.fit_transform(x_train)
x_test = RS.fit_transform(x_test)
x_val = RS.fit_transform(x_val)
# reshape
x_train = x_train.reshape(-1, 80, x_train.shape[-1])
x_test = x_test.reshape(-1, 80, x_test.shape[-1])
x_val = x_val.reshape(-1, 80, x_val.shape[-1])

In [ ]:
x_train.shape[-2:]

(80, 89)

In [ ]:
from torch.functional import split
# datasets.py: Define the data interator (data loader)
from scipy.ndimage import convolve1d
from scipy.ndimage import convolve1d, gaussian_filter1d
import os
import numpy as np
from torch.utils import data
from scipy.signal.windows import triang
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

def get_lds_kernel_window(kernel, ks, sigma):
    assert kernel in ['gaussian', 'triang', 'laplace']
    half_ks = (ks - 1) // 2
    if kernel == 'gaussian':
        base_kernel = [0.] * half_ks + [1.] + [0.] * half_ks
        kernel_window = gaussian_filter1d(base_kernel, sigma=sigma) / max(gaussian_filter1d(base_kernel, sigma=sigma))
    elif kernel == 'triang':
        kernel_window = triang(ks)
    else:
        laplace = lambda x: np.exp(-abs(x) / sigma) / (2. * sigma)
        kernel_window = list(map(laplace, np.arange(-half_ks, half_ks + 1))) / max(map(laplace, np.arange(-half_ks, half_ks + 1)))

    return kernel_window

def GetWeights(labels, reweight='sqrt_inv', max_target=65, lds=True, lds_kernel='gaussian',lds_ks=5,lds_sigma=2):
  
  value_dict = {x: 0 for x in range(-1,max_target)}
  labels = labels.numpy()
  labels = np.squeeze(labels.reshape(-1,1))
  
  #mbr
  for label in labels:
    value_dict[min(max_target-1, int(label))] += 1
  if reweight == 'sqrt_inv':
    value_dict = {k: np.sqrt(v) for k, v in value_dict.items()}
  elif reweight == 'inverse':
    value_dict = {k: np.clip(v, 5, 1000) for k, v in value_dict.items()}
  num_per_label = [value_dict[min(max_target-1, int(label))] for label in labels]
  if not len(num_per_label) or reweight == 'none':
    return None
  print(f"using re-weighting: [{reweight.upper()}]")
  if lds:
    lds_kernel_window = get_lds_kernel_window(lds_kernel, lds_ks, lds_sigma)
    print(f"Using LDS: [{lds_kernel.upper()}] ({lds_ks}/{lds_sigma})")
    smoother_value = convolve1d(np.asarray([v for _,v in value_dict.items()]), weights=lds_kernel_window, mode='constant')
    num_per_label = [smoother_value[min(max_target-1,int(label))] for label in labels]
  
  weights = [np.float64(1/x) for x in num_per_label]
  scaling = len(weights)/np.sum(weights)
  weights = [scaling * x for x in weights]
  return tf.convert_to_tensor(np.array(weights))

In [ ]:
# Model Configuration
class CFG:
    seed = 42
    VERBOSE = 1
    random_state = 42
    N_FOLDS = 5
    EPOCHS = 20
    BATCH_SIZE = 128
    factor = 0.5# Custom MAE Loss
    patience_1 = 5
    patience_2 = 15
    learning_rate = 1e-3
    weight_decay = 1e-3
    dropout_rate = 0.2

In [ ]:
# Accelerator Configuration
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

Running on 1 replicas
Batch Size: 512


In [ ]:
def custom_mae_loss(y_true, y_pred):
  w = GetWeights(y_true)
  mae = tf.reshape(w, [-1]) * tf.abs(tf.reshape(y_true, [-1]) - tf.reshape(y_pred, [-1]))
  return tf.reduce_sum(mae, axis=-1) #/ tf.reduce_sum(w, axis=-1)

In [ ]:
def dnn_model():
    x_input = Input(shape=(x_train.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=1024, return_sequences=True ))(x_input)
    c1 = concatenate([x_input, x1])
    
    x2 = Dropout(0.1)(c1)
    c2 = concatenate([x1, x2])

    x3= Bidirectional(LSTM(units=512, return_sequences=True))(c2)
    c3 = concatenate([x2, x3])
    
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(c3)
    c4 = concatenate([x3, x4])

    x5 = Dropout(0.2)(c4)
    c5 = concatenate([x4, x5])
    
    x6 = Bidirectional(LSTM(units=128, return_sequences=True))(c5)
    c6 = concatenate([x5, x6])

    x7 = Dropout(0.1)(c6)
    c7 = concatenate([x6, x7])
    
    x8 = Dense(units=128, activation='selu')(c7)
    x_output = Dense(units=1)(x8)
    
    model = Model(inputs=x_input, outputs=x_output, name='DNN_Model')
    model.compile(optimizer='Adam', loss=custom_mae_loss, run_eagerly=True)
    
    return model


In [ ]:
model = dnn_model()
lr = ReduceLROnPlateau(monitor="val_loss", factor=CFG.factor, patience=CFG.patience_1,
                               verbose=CFG.VERBOSE)
es = EarlyStopping(monitor="val_loss", patience=CFG.patience_2, mode="min",
                    restore_best_weights=True, verbose=CFG.VERBOSE)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=CFG.EPOCHS, batch_size=512,callbacks=[es])

Epoch 1/20
using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
1/9 [==>...........................] - ETA: 1:29 - loss: 683110.5625using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
2/9 [=====>........................] - ETA: 31s - loss: 545803.0625 using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
3/9 [=========>....................] - ETA: 27s - loss: 564060.4375using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
4/9 [============>.................] - ETA: 22s - loss: 507007.9688using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
5/9 [===============>..............] - ETA: 18s - loss: 473900.6875using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
6/9 [===================>..........] - ETA: 13s - loss: 446304.7500using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
7/9 [======================>.......] - ETA: 9s - loss: 428612.9688 using re-weighting: [SQRT_INV]
Using LDS: [GAUSSIAN] (5/2)
8/9 [=========================>....] - ETA: 4s

In [ ]:
model.save('/content/drive/MyDrive/Deep Learning đồ án/model/F0_LDS.h5')

In [ ]:
def mae_loss(y_true, y_pred):
  mae = tf.abs(tf.reshape(y_true, [-1]) - tf.reshape(y_pred, [-1]))
  return tf.reduce_sum(mae, axis=-1)

In [ ]:
def dnn_model():
    x_input = Input(shape=(x_train.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=1024, return_sequences=True ))(x_input)
    c1 = concatenate([x_input, x1])
    
    x2 = Dropout(0.1)(c1)
    c2 = concatenate([x1, x2])

    x3= Bidirectional(LSTM(units=512, return_sequences=True))(c2)
    c3 = concatenate([x2, x3])
    
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(c3)
    c4 = concatenate([x3, x4])

    x5 = Dropout(0.2)(c4)
    c5 = concatenate([x4, x5])
    
    x6 = Bidirectional(LSTM(units=128, return_sequences=True))(c5)
    c6 = concatenate([x5, x6])

    x7 = Dropout(0.1)(c6)
    c7 = concatenate([x6, x7])
    
    x8 = Dense(units=128, activation='selu')(c7)
    x_output = Dense(units=1)(x8)
    
    model = Model(inputs=x_input, outputs=x_output, name='DNN_Model')
    model.compile(optimizer='Adam', loss="mean_absolute_error", run_eagerly=True)
    
    return model


In [ ]:
model1 = dnn_model()
lr = ReduceLROnPlateau(monitor="val_loss", factor=CFG.factor, patience=CFG.patience_1,
                               verbose=CFG.VERBOSE)
es = EarlyStopping(monitor="val_loss", patience=CFG.patience_2, mode="min",
                    restore_best_weights=True, verbose=CFG.VERBOSE)
history = model1.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=CFG.EPOCHS, batch_size=128,callbacks=[es])

Epoch 1/20
36/36 [==============================] - 30s 701ms/step - loss: 4.1944 - val_loss: 2.4894
Epoch 2/20
36/36 [==============================] - 26s 717ms/step - loss: 2.0055 - val_loss: 1.4826
Epoch 3/20
36/36 [==============================] - 26s 733ms/step - loss: 1.4269 - val_loss: 1.2119
Epoch 4/20
36/36 [==============================] - 27s 752ms/step - loss: 1.2095 - val_loss: 1.0627
Epoch 5/20
36/36 [==============================] - 27s 756ms/step - loss: 1.0841 - val_loss: 0.9814
Epoch 6/20
36/36 [==============================] - 28s 767ms/step - loss: 1.0022 - val_loss: 0.9224
Epoch 7/20
36/36 [==============================] - 28s 778ms/step - loss: 0.9437 - val_loss: 0.8668
Epoch 8/20
36/36 [==============================] - 28s 777ms/step - loss: 0.9042 - val_loss: 0.8596
Epoch 9/20
36/36 [==============================] - 28s 777ms/step - loss: 0.8679 - val_loss: 0.8179
Epoch 10/20
36/36 [==============================] - 28s 780ms/step - loss: 0.8595 - val_lo

In [ ]:
model1.save('/content/drive/MyDrive/Deep Learning đồ án/model/F0.h5')

In [ ]:
y_pred = model.predict(x_test)

8/8 [==============================] - 1s 90ms/step


In [ ]:
print(r2_score(y_test, np.squeeze(y_pred)))
print(mean_absolute_error(y_test,np.squeeze(y_pred)))

0.9811113908674489
0.6601601163608335


In [ ]:
def short_metrics(preds, labels, train_labels, many_shot_thr=50000, low_shot_thr=1000):
  train_class_count, test_class_count = [], []
  mse_per_class, l1_per_class, r2_per_class = [], [], []
  for l in np.unique(np.floor(labels)):
    train_class_count.append(len(train_labels[np.floor(train_labels)==l]))
    test_class_count.append(len(labels[np.floor(labels)==l]))
    mse_per_class.append(np.sum((preds[np.floor(labels)==l]-labels[np.floor(labels)==l])**2))
    l1_per_class.append(np.sum(np.abs(preds[np.floor(labels)==l]-labels[np.floor(labels)==l])))
    r2_per_class.append(r2_score(labels[np.floor(labels)==l],preds[np.floor(labels)==l]))

  many_shot_mse, median_shot_mse, low_shot_mse = [], [], []
  many_shot_l1, median_shot_l1, low_shot_l1 = [], [], []
  many_shot_r2, median_shot_r2, low_shot_r2 = [], [], []
  many_shot_cnt, median_shot_cnt, low_shot_cnt = [], [], []
  for i in range(len(train_class_count)):
      if train_class_count[i] > many_shot_thr:
          many_shot_mse.append(mse_per_class[i])
          many_shot_l1.append(l1_per_class[i])
          many_shot_r2.append(r2_per_class[i])
          many_shot_cnt.append(test_class_count[i])
      elif train_class_count[i] < low_shot_thr:
          low_shot_mse.append(mse_per_class[i])
          low_shot_l1.append(l1_per_class[i])
          low_shot_r2.append(r2_per_class[i])
          low_shot_cnt.append(test_class_count[i])
      else:
          median_shot_mse.append(mse_per_class[i])
          median_shot_l1.append(l1_per_class[i])
          median_shot_r2.append(r2_per_class[i])
          median_shot_cnt.append(test_class_count[i])

  shot_dict = {}
  shot_dict['many mse'] = np.sum(many_shot_mse) / np.sum(many_shot_cnt)
  shot_dict['many l1'] = np.sum(many_shot_l1) / np.sum(many_shot_cnt)
  shot_dict['many r2'] = np.sum(many_shot_r2) / np.sum(many_shot_cnt)
  shot_dict['median mse'] = np.sum(median_shot_mse) / np.sum(median_shot_cnt)
  shot_dict['median l1'] = np.sum(median_shot_l1) / np.sum(median_shot_cnt)
  shot_dict['median r2'] = np.sum(median_shot_r2) / np.sum(median_shot_cnt)
  shot_dict['low mse'] = np.sum(low_shot_mse) / np.sum(low_shot_cnt)
  shot_dict['low l1'] = np.sum(low_shot_l1) / np.sum(low_shot_cnt)
  shot_dict['low r2'] = np.sum(low_shot_r2) / np.sum(low_shot_cnt)

  mse = mean_squared_error(preds, labels)
  mae = mean_absolute_error(preds, labels)
  r2 = r2_score(labels, preds)
  print(f" * Overall: MSE {mse:.3f}\tL1 {mae:.3f}\tR2 {r2:.3f}")
  print(f" * Many: MSE {shot_dict['many mse']:.3f}\t"
              f"L1 {shot_dict['many l1']:.3f}\tR2 score {shot_dict['many r2']:.3f}")
  print(f" * Median: MSE {shot_dict['median mse']:.3f}\t"
              f"L1 {shot_dict['median l1']:.3f}\tR2 score {shot_dict['median r2']:.3f}")
  print(f" * Low: MSE {shot_dict['low mse']:.3f}\t"
              f"L1 {shot_dict['low l1']:.3f}\tR2 score {shot_dict['low r2']:.3f}")

  return shot_dict

# shot_dict = short_metrics(y_pred.reshape(-1), y_test.reshape(-1), y_train.reshape(-1))

In [ ]:
np.unique(np.floor(y_train), return_counts=True)

(array([-2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,
        11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.,
        24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36.,
        37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49.,
        50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62.,
        63., 64.]),
 array([   144,     11,     66,    245,    385,   2871,   9891,  40277,
        123579,  38010,  14378,  12912,  10554,   7083,   6581,   6763,
          7060,   6410,   5132,   5048,   4634,   5256,   5353,   4098,
          3974,   3603,   4175,   4444,   3405,   2889,   2816,   3135,
          3162,   2455,   1730,   1590,   1691,   1834,    954,    658,
           470,    438,    426,    315,    221,    186,    176,    170,
           127,    117,     68,     61,     64,     49,     35,     52,
            32,     31,     23,     17,     20,     18,     13,     12,
            12,      8,     63]))

In [ ]:
model = load_model('/content/drive/MyDrive/Deep Learning đồ án/model/F0_LDS.h5')
y_pred = model.predict(x_test)
shot_dict = short_metrics(y_pred.reshape(-1), y_test.reshape(-1), y_train.reshape(-1))

ValueError: ignored

In [ ]:
# Result of plain model
shot_dict = short_metrics(y_pred.reshape(-1), y_test.reshape(-1), y_train.reshape(-1))

 * Overall: MSE 1.253	L1 0.660	R2 0.981
 * Many: MSE 0.111	L1 0.243	R2 score -0.000
 * Median: MSE 1.751	L1 0.854	R2 score -0.091
 * Low: MSE 5.948	L1 1.891	R2 score nan


In [ ]:
# LDS model
# y_pred = model.predict(x_test)
shot_dict = short_metrics(y_pred.reshape(-1), y_test.reshape(-1), y_train.reshape(-1))

 * Overall: MSE 2.504	L1 0.942	R2 0.963
 * Many: MSE 0.301	L1 0.351	R2 score -0.001
 * Median: MSE 3.539	L1 1.229	R2 score -0.190
 * Low: MSE 8.579	L1 2.204	R2 score nan


In [ ]:
# plain model
print(r2_score(y_test, np.squeeze(y_pred)))
print(mean_absolute_error(y_test,np.squeeze(y_pred)))

0.9811113908674489
0.6601601163608335


In [ ]:
mae = tf.keras.losses.MeanAbsoluteError(
    reduction=tf.keras.losses.Reduction.SUM)
mae(y_pred.reshape(-1), y_test.reshape(-1)).numpy()

0.9584600232089502

In [ ]:
model.save_weights("/content/drive/MyDrive/Deep Learning đồ án/model/weights_F0_LDS")
# load_status = model.load_weights("weights_F0_LDS")

In [ ]:
model = dnn_model()
load_status = model.load_weights("/content/drive/MyDrive/Deep Learning đồ án/model/weights_F0_LDS")

Using DenseWeight

In [ ]:
!pip install denseweight

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from denseweight import DenseWeight
def weight_mae_loss(y_true, y_pred):
  # y_true=tf.reshape(y_true, [-1])
  dw = DenseWeight(alpha=2)
  w = dw.fit(y_true.numpy().reshape(-1))
  tf.convert_to_tensor(w)
  mae = tf.reshape(w, [-1]) * tf.abs(tf.reshape(y_true, [-1]) - tf.reshape(y_pred, [-1]))
  return tf.reduce_sum(mae, axis=-1) #/ tf.reduce_sum(w, axis=-1)

In [ ]:
def dnn_model():
    x_input = Input(shape=(x_train.shape[-2:]), dtype='float32')
    
    x1 = Bidirectional(LSTM(units=1024, return_sequences=True))(x_input)
    c1 = concatenate([x_input, x1])
    
    x2 = Dropout(0.1)(c1)
    c2 = concatenate([x1, x2])

    x3= Bidirectional(LSTM(units=512, return_sequences=True))(c2)
    c3 = concatenate([x2, x3])
    
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(c3)
    c4 = concatenate([x3, x4])

    x5 = Dropout(0.2)(c4)
    c5 = concatenate([x4, x5])
    
    x6 = Bidirectional(LSTM(units=128, return_sequences=True))(c5)
    c6 = concatenate([x5, x6])

    x7 = Dropout(0.1)(c6)
    c7 = concatenate([x6, x7])
    
    x8 = Dense(units=128, activation='selu', )(c7)
    x_output = Dense(units=1)(x8)
    
    model = Model(inputs=x_input, outputs=x_output, name='DNN_Model')
    model.compile(optimizer='Adam', loss=weight_mae_loss, run_eagerly=True)
    
    return model


In [ ]:
x_train.dtype

dtype('float64')

In [ ]:
model = dnn_model()
lr = ReduceLROnPlateau(monitor="val_loss", factor=CFG.factor, patience=CFG.patience_1,
                               verbose=CFG.VERBOSE)
es = EarlyStopping(monitor="val_loss", patience=CFG.patience_2, mode="min",
                    restore_best_weights=True, verbose=CFG.VERBOSE)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=CFG.EPOCHS, batch_size=512,callbacks=[es])

Epoch 1/20


InternalError: ignored

In [ ]:
y_pred = model.predict(x_test)
shot_dict = short_metrics(y_pred.reshape(-1), y_test.reshape(-1), y_train.reshape(-1))

8/8 [==============================] - 1s 98ms/step
 * Overall: MSE 2.569	L1 1.002	R2 0.962
 * Many: MSE 0.559	L1 0.600	R2 score -0.001
 * Median: MSE 3.222	L1 1.160	R2 score -0.182
 * Low: MSE 19.716	L1 3.345	R2 score nan


In [ ]:
from collections import Counter
from scipy.ndimage import convolve1d
from utils import get_lds_kernel_window

# preds, labels: [Ns,], "Ns" is the number of total samples
preds, labels = ..., ...
# assign each label to its corresponding bin (start from 0)
# with your defined get_bin_idx(), return bin_index_per_label: [Ns,] 
bin_index_per_label = [get_bin_idx(label) for label in labels]

# calculate empirical (original) label distribution: [Nb,]
# "Nb" is the number of bins
Nb = max(bin_index_per_label) + 1
num_samples_of_bins = dict(Counter(bin_index_per_label))
emp_label_dist = [num_samples_of_bins.get(i, 0) for i in range(Nb)]

# lds_kernel_window: [ks,], here for example, we use gaussian, ks=5, sigma=2
lds_kernel_window = get_lds_kernel_window(kernel='gaussian', ks=5, sigma=2)
# calculate effective label distribution: [Nb,]
eff_label_dist = convolve1d(np.array(emp_label_dist), weights=lds_kernel_window, mode='constant')

array([1.00017848, 1.00051904, 0.99963178, ..., 0.99962069, 0.99959788,
       0.99978285])

In [ ]:
from loss import weighted_mse_loss

# Use re-weighting based on effective label distribution, sample-wise weights: [Ns,]
eff_num_per_label = [eff_label_dist[bin_idx] for bin_idx in bin_index_per_label]
weights = [np.float32(1 / x) for x in eff_num_per_label]

# calculate loss
loss = weighted_mse_loss(preds, labels, weights=weights)

In [ ]:
bins = args.bucket_num
        value_lst, bins_edges = np.histogram(targs, bins=bins, range=(0., 5.))